In [1]:
import urllib.request
import numpy as np
import pandas as pd
import re

In [2]:
# 网易也提供了比较完整的财务指标数据
server = "http://quotes.money.163.com/"
path = "service/"
table4 = "zycwzb_"
stock = "600585"    # 根据需求变更stock中的股票代码
type = ".html?type=year"
part5 = "&part=ylnl"
part6 = "&part=chnl"
part7 = "&part=cznl"
part8 = "&part=yynl"

target4 = server + path + table4 + stock + type
target5 = server + path + table4 + stock + type + part5
target6 = server + path + table4 + stock + type + part6
target7 = server + path + table4 + stock + type + part7
target8 = server + path + table4 + stock + type + part8

print(target4, '\n', target5, '\n', target6, '\n', target7, '\n', target8)

http://quotes.money.163.com/service/zycwzb_600585.html?type=year 
 http://quotes.money.163.com/service/zycwzb_600585.html?type=year&part=ylnl 
 http://quotes.money.163.com/service/zycwzb_600585.html?type=year&part=chnl 
 http://quotes.money.163.com/service/zycwzb_600585.html?type=year&part=cznl 
 http://quotes.money.163.com/service/zycwzb_600585.html?type=year&part=yynl


In [11]:
# 获取主要财务指标文件
df4 = pd.read_csv(target4, header=0, encoding="gb18030")    # 主要财务指标
df4
# 基本每股收益EPS，每股净资产=股东权益/总股数

,报告日期,2021-12-31,2020-12-31,2019-12-31,2018-12-31,2017-12-31,2016-12-31,2015-12-31,2014-12-31,2013-12-31,...,2007-12-31,2006-12-31,2005-12-31,2004-12-31,2003-12-31,2002-12-31,2001-12-31,2000-12-31,1999-12-31,Unnamed: 24
0,基本每股收益(元),6.28,6.63,6.34,5.63,2.99,1.61,1.42,2.07,1.77,...,1.70,--,--,--,--,--,--,--,--,NaN
1,每股净资产(元),34.66,30.54,25.92,21.26,16.87,14.46,13.30,12.50,10.59,...,7.07,5.62,4.55,4.35,3.65,2.72,2.25,2.07,1.98,NaN
2,每股经营活动产生的现金流量净额(元),6.40,6.57,7.69,6.80,3.28,2.49,1.87,3.33,2.87,...,1.70,2.33,1.21,1.14,1.98,0.77,0.74,0.48,0.26,NaN
3,主营业务收入(万元),16795266.00,17624268.00,15703033.00,12840263.00,7531082.00,5593190.00,5097604.00,6075850.00,5526168.00,...,1877610.00,1524628,1082621,838495,569503,299978,205835,123774,107483,NaN
4,主营业务利润(万元),4852800.00,5010642.00,5086719.00,4571493.00,2547610.00,1748949.00,1366292.00,2015418.00,1794681.00,...,571631.00,440692,244613,275205,234828,101926,73708,45806,37335,NaN
5,营业利润(万元),4310919.00,4627077.00,4405734.00,3988236.00,2082530.00,1076026.00,866651.00,1382348.00,1174268.00,...,317058.00,223751,80992,175725,155830,51426,28990,9909,2568,NaN
6,投资收益(万元),140193.00,154997.00,120908.00,65849.00,202669.00,39909.00,188090.00,-2141.00,-264.00,...,424.00,471,758,515,343,-134,47,1716,929,NaN
7,营业外收支净额(万元),100701.00,83715.00,49951.00,-25316.00,40346.00,89295.00,137289.00,105933.00,88859.00,...,31440.00,-739,-5431,51,-628,-1227,31,-152,8,NaN
8,利润总额(万元),4411620.00,4710792.00,4455685.00,3962920.00,2122876.00,1165321.00,1003940.00,1488281.00,1263127.00,...,348499.00,249831,86137,194173,164917,54373,31487,13190,4879,NaN
9,净利润(万元),3326707.00,3512969.00,3359276.00,2981428.00,1585467.00,852992.00,751639.00,1099302.00,938016.00,...,249422.00,142793,40689,100883,73957,26376,20273,11377,5859,NaN


In [4]:

df5 = pd.read_csv(target5, header=0, encoding="gb18030")    # 盈利能力
df5


,报告日期,2021-12-31,2020-12-31,2019-12-31,2018-12-31,2017-12-31,2016-12-31,2015-12-31,2014-12-31,2013-12-31,...,2007-12-31,2006-12-31,2005-12-31,2004-12-31,2003-12-31,2002-12-31,2001-12-31,2000-12-31,1999-12-31,Unnamed: 24
0,总资产利润率(%),14.82,18.1,19.22,20.49,13.45,8.17,7.21,11.33,10.54,...,8.71,8.28,3.05,9.03,8.35,4.17,4.36,2.57,0.92,NaN
1,主营业务利润率(%),28.89,28.43,32.39,35.60,33.83,31.27,26.80,33.17,32.48,...,30.44,27.84,22.59,32.82,41.23,33.98,35.81,37.01,34.74,NaN
2,总资产净利润率(%),15.84,19.15,20.93,22.55,14.18,8.31,7.33,11.86,10.86,...,10.02,9.01,3.33,9.87,10.01,4.96,5.03,2.6,0.92,NaN
3,成本费用利润率(%),34.85,35.43,39.09,44.18,37.19,25.78,22.73,31.71,29.13,...,22.33,18.81,8.60,29.32,39.86,21.89,17.85,11.7,4.67,NaN
4,营业利润率(%),25.67,26.25,28.06,31.06,27.65,19.24,17.00,22.75,21.25,...,16.89,14.31,8.46,23.15,29.07,18.53,15.28,10.78,4.53,NaN
5,主营业务成本率(%),70.37,70.84,66.71,63.26,64.91,67.53,72.36,66.27,66.99,...,68.84,71.50,76.80,66.58,58.05,65.38,63.49,62.22,64.5,NaN
6,销售净利率(%),20.34,20.64,21.88,23.86,21.81,16.00,14.96,19.07,17.75,...,14.40,11.80,5.43,17.31,19.54,12.38,12.85,9.2,3.69,NaN
7,净资产收益率(%),18.11,21.71,24.46,26.46,17.73,11.13,10.66,16.60,16.72,...,22.51,20.25,7.00,18.00,16.13,8.19,9.17,5.6,3.01,NaN
8,股本报酬率(%),1620.37,1582.71,1472.18,1324.00,975.49,751.86,645.52,644.59,544.62,...,526.89,509.79,329.46,338.80,268.93,182.18,--,--,--,NaN
9,净资产报酬率(%),44.76,49.86,54.82,60.27,56.21,49.62,46.28,49.07,49.11,...,71.47,72.17,60.36,64.08,59.20,53.54,58.45,55.51,47.48,NaN


In [8]:
df6 = pd.read_csv(target6, header=0, encoding="gb18030")    # 偿还能力
df6

,报告日期,2021-12-31,2020-12-31,2019-12-31,2018-12-31,2017-12-31,2016-12-31,2015-12-31,2014-12-31,2013-12-31,...,2007-12-31,2006-12-31,2005-12-31,2004-12-31,2003-12-31,2002-12-31,2001-12-31,2000-12-31,1999-12-31,Unnamed: 24
0,流动比率(%),3.78,4.66,3.54,2.76,2.18,1.72,1.28,1.74,1.69,...,0.73,0.62,0.46,0.59,0.65,0.40,0.59,0.65,0.61,NaN
1,速动比率(%),3.48,4.38,3.34,2.52,1.95,1.46,1.08,1.44,1.44,...,0.57,0.43,0.33,0.45,0.55,0.32,0.48,0.52,0.47,NaN
2,现金比率(%),212.85,256.69,200.59,143.85,117.27,91.42,70.93,98.30,77.39,...,14.77,19.59,16.01,26.82,45.45,22.82,33.48,28.22,23.34,NaN
3,利息支付倍数(%),-3255.85,-3009.90,-3229.69,-8258.99,9945.46,3559.40,1862.84,2183.00,1404.20,...,674.01,594.16,293.45,815.47,976.12,669.38,383.89,237.49,145.65,NaN
4,资产负债率(%),16.78,16.30,20.39,22.15,24.71,26.68,30.13,31.92,36.88,...,62.80,61.33,64.38,58.68,57.22,54.75,50.06,45.23,47.49,NaN
5,长期债务与营运资金比率(%),0.04,0.04,0.06,0.06,0.20,0.44,0.39,0.20,0.37,...,-3.76,-2.75,-1.23,-1.29,-1.48,-0.58,-1.40,-1.96,-1.57,NaN
6,股东权益比率(%),83.22,83.70,79.61,77.85,75.29,73.32,69.87,68.08,63.12,...,37.20,38.67,35.62,41.32,42.78,45.25,49.94,54.77,52.51,NaN
7,长期负债比率(%),1.63,1.65,2.17,1.74,3.98,4.97,2.12,2.06,4.01,...,31.64,31.26,25.34,20.26,19.49,13.93,18.10,18.29,18.15,NaN
8,股东权益与固定资产比率(%),283.45,262.46,234.82,191.15,153.13,128.13,119.76,121.49,113.94,...,61.09,58.98,52.77,62.38,74.13,72.15,82.51,89.71,78.78,NaN
9,负债与所有者权益比率(%),20.17,19.47,25.62,28.46,32.82,36.38,43.12,46.89,58.42,...,168.80,158.58,180.73,142.03,133.76,121.01,100.25,82.60,90.45,NaN


In [9]:
df7 = pd.read_csv(target7, header=0, encoding="gb18030")    # 成长能力
df7

,报告日期,2021-12-31,2020-12-31,2019-12-31,2018-12-31,2017-12-31,2016-12-31,2015-12-31,2014-12-31,2013-12-31,...,2007-12-31,2006-12-31,2005-12-31,2004-12-31,2003-12-31,2002-12-31,2001-12-31,2000-12-31,1999-12-31,Unnamed: 24
0,主营业务收入增长率(%),-4.70,12.23,22.30,70.50,34.65,9.72,-16.10,9.95,20.75,...,16.65,48.68,29.11,47.23,89.85,45.74,66.30,15.16,--,NaN
1,净利润增长率(%),-6.06,5.88,12.13,86.48,83.55,17.34,-34.17,18.10,51.83,...,42.36,223.34,-59.52,30.42,199.60,40.44,132.16,187.02,--,NaN
2,净资产增长率(%),14.04,18.20,22.25,26.59,14.53,8.64,6.18,18.46,14.83,...,30.19,29.41,3.25,16.39,41.65,32.89,24.62,6.94,--,NaN
3,总资产增长率(%),14.70,12.42,19.55,22.44,11.53,3.53,3.45,9.84,6.37,...,35.34,19.20,19.76,20.50,49.82,46.66,36.67,2.53,--,NaN


In [10]:
df8 = pd.read_csv(target8, header=0, encoding="gb18030")    # 营运能力
df8

,报告日期,2021-12-31,2020-12-31,2019-12-31,2018-12-31,2017-12-31,2016-12-31,2015-12-31,2014-12-31,2013-12-31,...,2007-12-31,2006-12-31,2005-12-31,2004-12-31,2003-12-31,2002-12-31,2001-12-31,2000-12-31,1999-12-31,Unnamed: 24
0,应收账款周转率(次),93.79,142.24,125.31,112.03,87.86,103.89,134.98,175.58,187.57,...,60.36,53.66,86.25,197.92,106.62,46.88,29.71,10.12,6.39,NaN
1,应收账款周转天数(天),3.84,2.53,2.87,3.21,4.10,3.47,2.67,2.05,1.92,...,5.96,6.71,4.17,1.82,3.38,7.68,12.12,35.56,56.31,NaN
2,存货周转率(次),13.99,19.86,18.07,15.14,10.57,8.60,8.56,9.98,9.58,...,9.12,10.29,9.09,8.44,8.69,7.61,7.11,4.71,4.06,NaN
3,固定资产周转率(次),2.55,2.83,2.58,2.12,1.23,0.90,0.86,1.12,1.10,...,1.11,1.15,0.92,0.91,0.86,0.65,0.64,0.44,0.25,NaN
4,总资产周转率(次),0.78,0.93,0.96,0.95,0.65,0.52,0.49,0.62,0.61,...,0.70,0.76,0.61,0.57,0.51,0.40,0.39,0.28,0.25,NaN
5,存货周转天数(天),25.74,18.13,19.92,23.77,34.07,41.87,42.03,36.07,37.59,...,39.49,34.99,39.61,42.64,41.45,47.32,50.61,76.36,88.69,NaN
6,总资产周转天数(天),462.43,387.85,376.33,380.87,553.68,692.84,734.54,578.69,588.33,...,517.46,471.70,587.08,631.14,702.71,898.20,918.84,1275.24,1450.44,NaN
7,流动资产周转率(次),1.42,1.68,1.86,2.18,2.00,2.01,1.99,2.44,2.31,...,3.35,4.21,3.09,2.47,2.47,2.34,2.14,1.60,139.79,NaN
8,流动资产周转天数(天),253.54,214.57,193.90,165.43,180.09,178.94,181.16,147.30,155.54,...,107.49,85.54,116.58,145.65,145.99,154.16,168.57,225.49,2.58,NaN
9,经营现金净流量对销售收入比率(%),0.20,0.20,0.26,0.28,0.23,0.24,0.19,0.29,0.28,...,0.14,0.19,0.14,0.17,0.44,0.30,0.36,0.38,0.24,NaN


#### 2.业绩预告

In [2]:
url2 = 'http://quotes.money.163.com/f10/yjyg_002270.html#01c03'
with urllib.request.urlopen(url2) as i:
    html = i.read()

data2 = pd.read_html(html)[3]    # 网页中每个预告独立成一个表，历史预告可以用4，5，6等依次提取
data2.[[0,1]]

,0,1
0,报告日期,2021-12-31
1,预测类型,预升
2,预测摘要,预计2021年1-12月归属于上市公司股东的净利润为：416943903.45元，与上年同期...
3,预测内容,预计2021年1-12月归属于上市公司股东的净利润为：416943903.45元，与上年同期...


In [62]:
# 从业绩预告中提取预增比例
prediction = data2.iloc[2, 1]
print(prediction)

预计2021年1-12月归属于上市公司股东的净利润为：416943903.45元，与上年同期相比变动幅度：45.13%。


In [79]:
forecast = re.findall(r"\d+\.?\d+%", prediction)    # 匹配百分数，可以是不带小数的。返回List
growrh_rate_estimate = float(forecast[0][0:-1]) / 100    #将list中的第一个string提取并丢弃最后一个百分号，转换成浮点数。
print(growrh_rate_estimate)


0.45130000000000003


In [44]:
#re.findall(r'\b\d+(?:\.\d+)?%?(?!\S)', prediction)
print(re.findall(r"\d+\.\d+%", prediction))

['45.13%', '45.13%', '17.71%', '3.35%', '9.25%']
